In [1]:
%load_ext autoreload
%autoreload 2
import time
from tweepy import TweepError
from requests.exceptions import Timeout, ConnectionError
import os
import tweepy
import json
import requests
import shutil

# THE CELL BELOW CONTAINS THE FILENAME CONSTANTS THAT NEED TO BE CHANGED

In [2]:
'''
    SOME QUICK HOW TO!
    
    To make this work, make sure you:
        - Have created an output folder - python won't do it if you haven't.
        - Have a lot of space and time, because this will probably take a long, long time.
        - Maybe have some kind of check for what happens after 15 minutes of pulling data,
            I have not tested it for that limit.
'''

# This file should be the one with the tweets as jsons in it.
TWEET_ID_FILE = "C:/Users/mfren/Desktop/irma_extras.json"

# The downloaded images should go into a folder.
OUTPUT_DIR = "C:/Users/mfren/Desktop/irma_images/"
ERROR_LOG = "C:/Users/mfren/Desktop/irma_images/error_log.txt"
retry_group = []

In [3]:
def exists(path):
  r = requests.head(path)
  return r.status_code == requests.codes.ok

In [4]:
with open(TWEET_ID_FILE) as f:
    all_content_as_string = f.read()

In [5]:
# split the file into a list of strings.
list_of_dicts = all_content_as_string.splitlines()

# turn each string into a json object
list_of_jsons = [json.loads(string) for string in list_of_dicts]

In [ ]:
# Go through the whole list of tweets and download the images.

image_count = 0
non_image_count = 0

for i in range(0, len(list_of_jsons)):

    # If there is media attached to the tweet.
    if 'extended_entities' in list_of_jsons[i]:
        # Make a temp (a little cleaner than getting it constantly)
        currentJson = dict(list_of_jsons[i])

        # Ok, iterate through each link in the media list.
        for j in range(0, len(currentJson['entities']['media'])):

            # Assign the URL for the media-piece.
            currentUrl = currentJson['entities']['media'][j]['media_url_https']

            # Check if the picture URL works and if the photo is indeed, a photo
            try:
                if (exists(currentUrl) and currentJson['entities']['media'][j]['type'] == 'photo'):
                    # Do the raw stream magic (Who knows how this works)
                    url_raw = requests.get(currentUrl, stream=True)
                    url_raw.raw.decode_content = True

                    # NAMING SCHEME: tweetID + userID + index of img
                    with open(OUTPUT_DIR + str(currentJson['id']) + '_' +
                              str(currentJson['user']['id']) + "_" + str(j) + ".jpg", 'wb+') as fin:
                        shutil.copyfileobj(url_raw.raw, fin)

                    # Remove the image url response object - because you should.
                    del url_raw

                    image_count += 1
            except requests.exceptions.RequestException as e:
                retry_group.append(currentJson)
    else:
        non_image_count += 1


# Repeat the same process for the retry group until the retry list is empty:
while len(retry_group) != 0:
    print("Running through the retry group now... ", len(retry_group), " items in the retry group: ")

    # If there is media attached to the tweet.
    if 'extended_entities' in retry_group[0]:
        # Make a temp (a little cleaner than getting it constantly)
        currentJson = dict(retry_group[0])
        retry_group.pop(0) # Remove the current json from the retry list

        # Ok, iterate through each link in the media list.
        for j in range(0, len(currentJson['entities']['media'])):

            # Assign the URL for the media-piece.
            currentUrl = currentJson['entities']['media'][j]['media_url_https']

            # Check if the picture URL works and if the photo is indeed, a photo
            try:
                if (exists(currentUrl) and currentJson['entities']['media'][j]['type'] == 'photo'):
                    # Do the raw stream magic (Who knows how this works)
                    url_raw = requests.get(currentUrl, stream=True)
                    url_raw.raw.decode_content = True

                    # NAMING SCHEME: tweetID + userID + index of img
                    with open(OUTPUT_DIR + str(currentJson['id']) + '_' +
                              str(currentJson['user']['id']) + "_" + str(j) + ".jpg", 'wb+') as fin:
                        shutil.copyfileobj(url_raw.raw, fin)

                    # Remove the image url response object - because you should.
                    del url_raw

                    image_count += 1
            except requests.exceptions.RequestException as e: # If another error occurs, add it back to the retry list
                retry_group.append(currentJson)
    else:
        non_image_count += 1


# So you know once it is done, + some S T A T I S T I C S
print("Found and downloaded", image_count, " images and", non_image_count, " other tweets. Total: ", i)